In [1]:
# 🔧 그룹 1: 설정 + 함수 정의
# (이 셀을 실행하면 모든 함수가 정의됩니다. 크롤링은 시작되지 않음)

import pandas as pd
import warnings, os, time, shutil, urllib, random
warnings.filterwarnings(action='ignore')

import re                        # 가격/평점 정제용 정규식
import json                      # 메타데이터 JSON 저장용  
from datetime import datetime    # 타임스탬프용

from PIL import Image
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException

import chromedriver_autoinstaller
import undetected_chromedriver as uc
from user_agents import parse
import selenium

print(f"🔧 Selenium 버전: {selenium.__version__}")

# ⭐⭐⭐ 중요 설정: 여기서 수정하세요! ⭐⭐⭐
# 기존 CONFIG 부분을 이것으로 교체
CONFIG = {
    "WAIT_TIMEOUT": 10,
    "RETRY_COUNT": 3,
    "MIN_DELAY": 5,                  # 3 → 5초로 증가
    "MAX_DELAY": 12,                 # 8 → 12초로 증가
    "POPUP_WAIT": 5,
    "SAVE_IMAGES": True,
    "SAVE_INTERMEDIATE": True,
    "MAX_PRODUCT_NAME_LENGTH": 30,
    "LONGER_DELAYS": True,           # 새로 추가
    "MEMORY_CLEANUP_INTERVAL": 5,    # 새로 추가
    "MAX_PRODUCTS_PER_CITY": 10,     # 2 → 10개로 증가⭐⭐⭐⭐⭐⭐⭐⭐⭐
}

# 🏙️ 검색할 도시들 (여기서 변경!)
CITIES_TO_SEARCH = ["오사카"]

# 도시별 대륙 및 국가 정보 매핑
CITY_INFO = {
    "방콕": {"대륙": "아시아", "국가": "태국"},
    "도쿄": {"대륙": "아시아", "국가": "일본"},
    "오사카": {"대륙": "아시아", "국가": "일본"},
    "싱가포르": {"대륙": "아시아", "국가": "싱가포르"},
    "홍콩": {"대륙": "아시아", "국가": "홍콩"},
    "파리": {"대륙": "유럽", "국가": "프랑스"},
    "런던": {"대륙": "유럽", "국가": "영국"},
    "뉴욕": {"대륙": "북미", "국가": "미국"},
    "시드니": {"대륙": "오세아니아", "국가": "호주"},
}
# 기존 CITY_INFO 코드는 그대로 두고...

# 🆕 여기에 추가!
CITY_CODES = {
    # 동남아시아
    "방콕": "BKK",
    "치앙마이": "CNX", 
    "푸켓": "HKT",
    "싱가포르": "SIN",
    "홍콩": "HKG",
    "쿠알라룸푸르": "KUL",
    "세부": "CEB",
    "다낭": "DAD",
    "호치민": "SGN",
    
    # 일본
    "도쿄": "NRT",
    "오사카": "KIX",
    "나고야": "NGO",
    "후쿠오카": "FUK",
    "오키나와": "OKA",
    "삿포로": "CTS",
    
    # 한국
    "서울": "ICN",
    "부산": "PUS",
    "제주": "CJU",
    "대구": "TAE",
    "광주": "KWJ",
    "여수": "RSU",
    
    # 유럽
    "파리": "CDG",
    "런던": "LHR",
    "로마": "FCO",
    "바르셀로나": "BCN",
    
    # 북미
    "뉴욕": "JFK",
    "로스앤젤레스": "LAX",
    "시카고": "ORD",
    
    # 오세아니아
    "시드니": "SYD",
    "멜버른": "MEL",
}

print(f"✅ CITY_CODES 추가 완료! {len(CITY_CODES)}개 도시 지원")

# 🚀 Phase 2: 확장성 개선 - 도시 데이터 파일 분리
# 기존 그룹 1에 추가할 코드들

# ================================================================================
# 📁 Step 1: 도시 데이터 파일 시스템 추가
# ================================================================================

# 기존 그룹 1의 CITY_CODES 아래에 이 함수들을 추가하세요!

def create_city_codes_file():
    """도시 코드를 JSON 파일로 저장"""
    
    # 기존 CITY_CODES를 파일로 저장
    city_data = {
        "version": "1.0",
        "last_updated": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        "cities": CITY_CODES,
        "total_cities": len(CITY_CODES)
    }
    
    try:
        with open('city_codes.json', 'w', encoding='utf-8') as f:
            json.dump(city_data, f, ensure_ascii=False, indent=2)
        print(f"✅ city_codes.json 파일 생성 완료! ({len(CITY_CODES)}개 도시)")
        return True
    except Exception as e:
        print(f"❌ 파일 생성 실패: {e}")
        return False

def load_city_codes_from_file():
    """JSON 파일에서 도시 코드 로드"""
    
    if not os.path.exists('city_codes.json'):
        print("📝 city_codes.json 파일이 없어서 새로 생성합니다...")
        create_city_codes_file()
        return CITY_CODES
    
    try:
        with open('city_codes.json', 'r', encoding='utf-8') as f:
            city_data = json.load(f)
        
        loaded_codes = city_data.get('cities', {})
        print(f"✅ city_codes.json 로드 완료! ({len(loaded_codes)}개 도시)")
        print(f"📅 마지막 업데이트: {city_data.get('last_updated', '알 수 없음')}")
        
        return loaded_codes
        
    except Exception as e:
        print(f"⚠️ 파일 로드 실패: {e}")
        print("💡 기존 코드의 CITY_CODES를 사용합니다.")
        return CITY_CODES

def add_new_city(city_name, airport_code, update_file=True):
    """새로운 도시를 추가하는 함수"""
    
    global CITY_CODES
    
    # 메모리에 추가
    CITY_CODES[city_name] = airport_code
    print(f"✅ 메모리에 추가: {city_name} → {airport_code}")
    
    # 파일에도 저장
    if update_file:
        if os.path.exists('city_codes.json'):
            try:
                # 기존 파일 읽기
                with open('city_codes.json', 'r', encoding='utf-8') as f:
                    city_data = json.load(f)
                
                # 새 도시 추가
                city_data['cities'][city_name] = airport_code
                city_data['total_cities'] = len(city_data['cities'])
                city_data['last_updated'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                
                # 파일에 저장
                with open('city_codes.json', 'w', encoding='utf-8') as f:
                    json.dump(city_data, f, ensure_ascii=False, indent=2)
                
                print(f"✅ city_codes.json 파일 업데이트 완료!")
                
            except Exception as e:
                print(f"⚠️ 파일 업데이트 실패: {e}")
        else:
            # 파일이 없으면 새로 생성
            create_city_codes_file()
    
    return True

def show_supported_cities():
    """지원하는 도시 목록 표시"""
    
    print("\n🌍 지원하는 도시 목록:")
    print("="*50)
    
    # 지역별로 분류
    regions = {
        "동남아시아": ["방콕", "치앙마이", "푸켓", "싱가포르", "홍콩", "쿠알라룸푸르", "세부", "다낭", "호치민"],
        "일본": ["도쿄", "오사카", "나고야", "후쿠오카", "오키나와", "삿포로"],
        "한국": ["서울", "부산", "제주", "대구", "광주", "여수"],
        "유럽": ["파리", "런던", "로마", "바르셀로나"],
        "북미": ["뉴욕", "로스앤젤레스", "시카고"],
        "오세아니아": ["시드니", "멜버른"]
    }
    
    for region, cities in regions.items():
        print(f"\n📍 {region}:")
        for city in cities:
            if city in CITY_CODES:
                code = CITY_CODES[city]
                print(f"   {city} → {code}")
    
    print(f"\n📊 총 {len(CITY_CODES)}개 도시 지원")
    print("="*50)

def update_config_for_scalability():
    """확장성을 위한 CONFIG 업데이트"""
    
    global CONFIG
    
    # 기존 CONFIG에 확장성 설정 추가
    scalability_config = {
        # 도시 관리
        "AUTO_LOAD_CITIES": True,           # 시작 시 파일에서 도시 자동 로드
        "AUTO_SAVE_NEW_CITIES": True,       # 새 도시 추가 시 파일 자동 저장
        
        # 다중 도시 지원
        "ENABLE_MULTI_CITY": False,         # 여러 도시 동시 크롤링 (나중에 사용)
        "CITY_PROCESSING_ORDER": "sequential",  # 도시 처리 순서
        
        # 파일 관리
        "BACKUP_OLD_DATA": True,            # 기존 데이터 백업
        "DATA_RETENTION_DAYS": 30,          # 데이터 보관 기간
        
        # 확장 기능
        "ENABLE_CITY_VALIDATION": True,     # 도시명 유효성 검사
        "ENABLE_DUPLICATE_CHECK": True,     # 중복 도시 체크
    }
    
    CONFIG.update(scalability_config)
    print("⚙️ CONFIG 확장성 설정 업데이트 완료!")

# ================================================================================
# 📁 Step 2: 파일 시스템 초기화
# ================================================================================

def initialize_file_system():
    """파일 시스템 초기화 및 설정"""
    
    print("🔧 Phase 2: 확장성 개선 시스템 초기화...")
    
    # CONFIG 업데이트
    update_config_for_scalability()
    
    # 도시 코드 파일 로드/생성
    if CONFIG.get("AUTO_LOAD_CITIES", True):
        global CITY_CODES
        loaded_codes = load_city_codes_from_file()
        
        # 새로 로드된 코드가 더 많으면 업데이트
        if len(loaded_codes) >= len(CITY_CODES):
            CITY_CODES = loaded_codes
            print(f"🔄 CITY_CODES 업데이트: {len(CITY_CODES)}개 도시")
        else:
            # 메모리의 코드가 더 최신이면 파일 업데이트
            create_city_codes_file()
    
    print("✅ Phase 2 시스템 초기화 완료!")
    return True

# ================================================================================
# 📁 Step 3: 사용자 편의 함수들
# ================================================================================

def quick_add_cities():
    """자주 사용하는 도시들을 빠르게 추가"""
    
    quick_cities = {
        # 추가로 자주 사용될 도시들
        "교토": "KIX",      # 오사카 공항 사용
        "인천": "ICN",      # 서울 공항
        "김포": "GMP",      # 김포공항
        "하와이": "HNL",    # 호놀룰루
        "괌": "GUM",        # 괌 국제공항
        "사이판": "SPN",    # 사이판 공항
        "푸꾸옥": "PQC",    # 푸꾸옥 공항
        "나트랑": "CXR",    # 나트랑 공항
        "보홀": "TAG",      # 보홀 공항
        "랑카위": "LGK",    # 랑카위 공항
    }
    
    print("🚀 자주 사용하는 도시들 추가 중...")
    
    for city, code in quick_cities.items():
        if city not in CITY_CODES:
            add_new_city(city, code, update_file=False)
    
    # 한 번에 파일 저장
    create_city_codes_file()
    
    print(f"✅ {len(quick_cities)}개 도시 일괄 추가 완료!")

def validate_city(city_name):
    """도시명 유효성 검사"""
    
    if not city_name or len(city_name.strip()) == 0:
        return False, "도시명이 비어있습니다."
    
    if city_name in CITY_CODES:
        return True, f"지원하는 도시입니다. ({CITY_CODES[city_name]})"
    
    # 유사한 도시명 찾기
    similar_cities = []
    for supported_city in CITY_CODES.keys():
        if city_name.lower() in supported_city.lower() or supported_city.lower() in city_name.lower():
            similar_cities.append(supported_city)
    
    if similar_cities:
        return False, f"지원하지 않는 도시입니다. 비슷한 도시: {', '.join(similar_cities)}"
    else:
        return False, f"지원하지 않는 도시입니다. 새로 추가하시려면 add_new_city() 함수를 사용하세요."

# ================================================================================
# 📁 Step 4: Phase 2 시스템 실행
# ================================================================================

# 자동 초기화 실행
try:
    initialize_file_system()
    
    # 자주 사용하는 도시들도 추가
    quick_add_cities()
    
    # 지원 도시 목록 표시
    show_supported_cities()
    
    print("\n🎉 Phase 2: 확장성 개선 완료!")
    print("💡 이제 이런 기능들을 사용할 수 있습니다:")
    print("   - add_new_city('제주도', 'CJU')  # 새 도시 추가")
    print("   - show_supported_cities()        # 지원 도시 목록")
    print("   - validate_city('방콕')          # 도시 유효성 검사")
    
except Exception as e:
    print(f"❌ Phase 2 초기화 실패: {e}")
    print("💡 기존 방식으로 계속 사용 가능합니다.")


# ===== 모든 함수들 정의 =====

def print_progress(current, total, city_name, status="진행중"):
    """진행률을 시각적으로 표시하는 함수"""
    percentage = (current / total) * 100
    bar_length = 30
    filled_length = int(bar_length * current // total)
    bar = '█' * filled_length + '░' * (bar_length - filled_length)
    
    emoji = "🔍" if status == "진행중" else "✅" if status == "완료" else "❌"
    
    print(f"\n{emoji} 진행률: [{bar}] {percentage:.1f}% ({current}/{total})")
    print(f"📍 현재 작업: {city_name} - {status}")
    print("-" * 50)

def print_product_progress(current, total, product_name):
    """상품별 진행률 표시 함수"""
    percentage = (current / total) * 100
    bar_length = 20
    filled_length = int(bar_length * current // total)
    bar = '█' * filled_length + '░' * (bar_length - filled_length)
    
    safe_name = str(product_name)[:30] + "..." if len(str(product_name)) > 30 else str(product_name)
    print(f"    🎯 상품 진행률: [{bar}] {percentage:.1f}% ({current}/{total})")
    print(f"    📦 현재 상품: {safe_name}")

def save_intermediate_results(results, city_name):
    """중간 결과를 임시 파일로 저장하는 함수"""
    if results and CONFIG["SAVE_INTERMEDIATE"]:
        try:
            timestamp = time.strftime('%Y%m%d_%H%M%S')
            temp_filename = f"temp_중간저장_{city_name}_{timestamp}.csv"
            pd.DataFrame(results).to_csv(temp_filename, index=False, encoding='utf-8-sig')
            print(f"  💾 중간 결과 저장: {temp_filename}")
            return temp_filename
        except Exception as e:
            print(f"  ⚠️ 중간 저장 실패: {e}")
            return None
    return None

def retry_operation(func, operation_name, max_retries=None):
    """실패한 작업을 재시도하는 함수"""
    if max_retries is None:
        max_retries = CONFIG["RETRY_COUNT"]
    
    for attempt in range(max_retries):
        try:
            return func()
        except (TimeoutException, NoSuchElementException, WebDriverException) as e:
            if attempt == max_retries - 1:
                print(f"  ❌ {operation_name} 최종 실패: {type(e).__name__}")
                raise e
            print(f"  🔄 {operation_name} 재시도 {attempt + 1}/{max_retries} (오류: {type(e).__name__})")
            time.sleep(2)
        except Exception as e:
            print(f"  ❌ {operation_name} 예상치 못한 오류: {type(e).__name__}: {e}")
            raise e

def get_city_info(city_name):
    """도시명으로 대륙과 국가 정보를 가져오는 함수"""
    info = CITY_INFO.get(city_name, {"대륙": "기타", "국가": "기타"})
    return info["대륙"], info["국가"]

def make_safe_filename(filename):
    """파일명에 사용할 수 없는 문자 제거"""
    if not filename:
        return "기본파일명"
    
    safe_filename = str(filename)
    unsafe_chars = ['/', '\\', ':', '*', '?', '"', '<', '>', '|', '\n', '\r', '\t']
    for char in unsafe_chars:
        safe_filename = safe_filename.replace(char, '_')
    
    if len(safe_filename) > 200:
        safe_filename = safe_filename[:200]
    
    if safe_filename.startswith('.'):
        safe_filename = '_' + safe_filename[1:]
    
    return safe_filename

def make_user_agent(ua, is_mobile):
    user_agent = parse(ua)
    model = user_agent.device.model
    platform = user_agent.os.family
    platform_version = user_agent.os.version_string + ".0.0"
    version = user_agent.browser.version[0]
    ua_full_version = user_agent.browser.version_string
    architecture = "x86"
    print(platform)
    if is_mobile:
        platform_info = "Linux armv8l"
        architecture= ""
    else:
        platform_info = "Win32"
        model = ""
    RET_USER_AGENT = {
        "appVersion" : ua.replace("Mozilla/", ""),
        "userAgent": ua,
        "platform" : f"{platform_info}",
        "acceptLanguage" : "ko-KR, kr, en-US, en",
        "userAgentMetadata":{
            "brands" : [
                {"brand":"Google Chrome", "version":f"{version}"},
                {"brand":"Chromium", "version":f"{version}"},
                {"brand":" Not A;Brand", "version":"99"}
            ],
            "fullVersionList" : [
                {"brand":"Google Chrome", "version":f"{version}"},
                {"brand":"Chromium", "version":f"{version}"},
                {"brand":" Not A;Brand", "version":"99"}
            ],
            "fullVersion":f"{ua_full_version}",
            "platform" :platform,
            "platformVersion":platform_version,
            "architecture":architecture,
            "model" : model,
            "mobile":is_mobile
        }
    }
    return RET_USER_AGENT

def generate_random_geolocation():
    ltop_lat = 37.75415601640249
    ltop_long = 126.86767642302573
    rbottom_lat = 37.593829172663945
    rbottom_long = 127.15276051439332

    targetLat = random.uniform(rbottom_lat, ltop_lat)
    targetLong = random.uniform(ltop_long,rbottom_long)
    return {"latitude":targetLat, "longitude" : targetLong, "accuracy":100}

def setup_driver():
    """크롬 드라이버 설정 및 실행"""
    chromedriver_autoinstaller.install()
    
    options = uc.ChromeOptions()
    
    UA = CONFIG["USER_AGENT"]
    options.add_argument(f"--user-agent={UA}")
    
    rand_user_folder = random.randrange(1,100)
    raw_path = os.path.abspath("cookies")
    try:
        shutil.rmtree(raw_path)
    except:
        pass
    os.makedirs(raw_path, exist_ok=True)
    user_cookie_name = f"{raw_path}/{rand_user_folder}"
    if os.path.exists(user_cookie_name) == False:
        os.makedirs(user_cookie_name, exist_ok=True)
    
    try:
        driver = uc.Chrome(user_data_dir=user_cookie_name, options=options)
        print("✅ 크롬 드라이버 실행 성공!")
    except Exception as e:
        print('\n',"-"*50,"\n","-"*50,"\n")
        print("# 키홈 메세지 : 혹시 여기서 에러 발생시 [아래 블로그 참고 -> 재부팅 -> 다시 코드실행] 해보시길 바랍니다! \n (구글크롬 버젼 업그레이드 문제)")
        print('https://appfollow.tistory.com/102')
        print('\n',"-"*50,"\n","-"*50,"\n")
        raise RuntimeError
        
    UA_Data = make_user_agent(UA,False)
    driver.execute_cdp_cmd("Network.setUserAgentOverride",UA_Data)
    
    GEO_DATA = generate_random_geolocation()
    driver.execute_cdp_cmd("Emulation.setGeolocationOverride", GEO_DATA)
    driver.execute_cdp_cmd("Emulation.setUserAgentOverride", UA_Data)
    driver.execute_cdp_cmd("Emulation.setNavigatorOverrides",{"platform":"Linux armv8l"})
    
    return driver

def go_to_main_page(driver):
    """메인 페이지로 이동"""
    driver.get("https://www.myrealtrip.com/experiences/")
    time.sleep(random.uniform(CONFIG["MIN_DELAY"], CONFIG["MAX_DELAY"]))
    return True

def find_and_fill_search(driver, city_name):
    """검색창 찾기 및 입력"""
    print(f"  🔍 '{city_name}' 검색창 찾는 중...")
    search_selectors = [
        (By.CSS_SELECTOR, "input[placeholder*='어디로']"),
        (By.CSS_SELECTOR, "input[type='text']"),
        (By.XPATH, "//input[contains(@placeholder, '어디로')]"),
        (By.XPATH, "/html/body/main/div/div[2]/section[1]/div[1]/div/div/input")
    ]

    search_input = None
    for selector_type, selector_value in search_selectors:
        try:
            search_input = WebDriverWait(driver, CONFIG["WAIT_TIMEOUT"]).until(
                EC.presence_of_element_located((selector_type, selector_value))
            )
            print(f"  ✅ 검색창을 찾았습니다!")
            break
        except TimeoutException:
            continue

    if not search_input:
        raise NoSuchElementException("검색창을 찾을 수 없습니다")

    search_input.clear()
    search_input.send_keys(city_name)
    time.sleep(random.uniform(CONFIG["MIN_DELAY"], CONFIG["MIN_DELAY"]+2))
    print(f"  📝 '{city_name}' 키워드 입력 완료")
    return True

def click_search_button(driver):
    """검색 버튼 클릭"""
    print(f"  🔎 검색 버튼 찾는 중...")
    search_button_selectors = [
        (By.CSS_SELECTOR, "button[type='submit']"),
        (By.CSS_SELECTOR, ".search-btn"),
        (By.XPATH, "//button[contains(@class, 'search')]"),
        (By.XPATH, "//img[contains(@alt, '검색')]//parent::*"),
        (By.XPATH, "/html/body/main/div/div[2]/section[1]/div[1]/div/div/div/img")
    ]

    search_clicked = False
    for selector_type, selector_value in search_button_selectors:
        try:
            search_button = WebDriverWait(driver, CONFIG["WAIT_TIMEOUT"]).until(
                EC.element_to_be_clickable((selector_type, selector_value))
            )
            search_button.click()
            print(f"  ✅ 검색 버튼 클릭 성공!")
            search_clicked = True
            time.sleep(random.uniform(CONFIG["MIN_DELAY"], CONFIG["MAX_DELAY"]))
            break
        except TimeoutException:
            continue

    if not search_clicked:
        raise NoSuchElementException("검색 버튼을 찾을 수 없습니다")
    return True

def handle_popup(driver):
    """팝업 처리"""
    popup_selectors = [
        (By.CSS_SELECTOR, ".popup-close"),
        (By.CSS_SELECTOR, ".modal-close"),
        (By.XPATH, "//button[contains(@aria-label, '닫기')]"),
        (By.XPATH, "//button[contains(text(), '닫기')]"),
        (By.XPATH, "/html/body/div[15]/div[2]/button")
    ]

    popup_closed = False
    for selector_type, selector_value in popup_selectors:
        try:
            popup_button = WebDriverWait(driver, CONFIG["POPUP_WAIT"]).until(
                EC.element_to_be_clickable((selector_type, selector_value))
            )
            popup_button.click()
            print(f"  ✅ 팝업창을 닫았습니다.")
            popup_closed = True
            time.sleep(random.uniform(1, 4))
            break
        except TimeoutException:
            continue

    if not popup_closed:
        print(f"  ℹ️ 팝업창이 없거나 이미 닫혀있습니다.")
    return True

def click_view_all(driver):
    """전체 상품 보기 버튼 클릭"""
    print(f"  📋 전체 상품 보기 버튼 찾는 중...")
    view_all_selectors = [
        (By.XPATH, "//button[contains(text(), '전체')]"),
        (By.XPATH, "//span[contains(text(), '전체')]//parent::button"),
        (By.CSS_SELECTOR, "button[aria-label*='전체']"),
        (By.XPATH, "/html/body/div[4]/div[2]/div/div/div/span[21]/button")
    ]

    view_all_clicked = False
    for selector_type, selector_value in view_all_selectors:
        try:
            view_all_button = WebDriverWait(driver, CONFIG["WAIT_TIMEOUT"]).until(
                EC.element_to_be_clickable((selector_type, selector_value))
            )
            view_all_button.click()
            print(f"  ✅ 전체 상품 보기 클릭 성공!")
            view_all_clicked = True
            time.sleep(random.uniform(CONFIG["MIN_DELAY"], CONFIG["MIN_DELAY"]+3))
            break
        except TimeoutException:
            continue

    if not view_all_clicked:
        print(f"  ⚠️ 전체 상품 보기 버튼을 찾을 수 없습니다. 현재 상품으로 진행...")
    return True

def collect_page_urls(driver):
    """현재 페이지의 모든 상품 URL 수집"""
    print(f"  📊 현재 페이지의 상품 URL들을 수집 중...")
    
    time.sleep(random.uniform(3, 5))
    
    product_url_selectors = [
        "a[href*='/experiences/']",
        "a[href*='/experience/']",
        ".product-item a",
        ".experience-card a"
    ]
    
    collected_urls = []
    
    for selector in product_url_selectors:
        try:
            product_elements = driver.find_elements(By.CSS_SELECTOR, selector)
            
            for element in product_elements:
                try:
                    url = element.get_attribute('href')
                    if url and '/experiences/' in url and url not in collected_urls:
                        collected_urls.append(url)
                except Exception as e:
                    continue
            
            if collected_urls:
                break
                
        except Exception as e:
            continue
    
    valid_urls = []
    for url in collected_urls:
        if url and url.startswith('http') and '/experiences/' in url:
            valid_urls.append(url)
    
    print(f"  ✅ {len(valid_urls)}개의 상품 URL을 수집했습니다!")
    
    if len(valid_urls) == 0:
        print("  ⚠️ 상품 URL을 찾을 수 없습니다. 페이지 구조를 확인해주세요.")
    
    return valid_urls

def get_product_name(driver):
    """상품명 수집"""
    print(f"  📊 상품 정보 수집 중...")
    title_selectors = [
        (By.CSS_SELECTOR, "h1"),
        (By.CSS_SELECTOR, ".product-title"),
        (By.XPATH, "//h1[contains(@class, 'title')]"),
        (By.XPATH, "/html/body/div[1]/main/div[1]/section/div[1]/h1")
    ]

    for selector_type, selector_value in title_selectors:
        try:
            title_element = WebDriverWait(driver, CONFIG["WAIT_TIMEOUT"]).until(
                EC.presence_of_element_located((selector_type, selector_value))
            )
            found_name = title_element.text
            time.sleep(random.uniform(CONFIG["MIN_DELAY"], CONFIG["MIN_DELAY"]+2))
            return found_name
        except TimeoutException:
            continue
    
    raise NoSuchElementException("상품명을 찾을 수 없습니다")

def get_price(driver):
    """가격 정보 수집"""
    price_selectors = [
        (By.CSS_SELECTOR, ".price"),
        (By.CSS_SELECTOR, "[class*='price']"),
        (By.XPATH, "//span[contains(text(), '원')]"),
        (By.XPATH, "/html/body/div[1]/main/div[1]/div[4]/div/div/div[2]/span[2]")
    ]

    for selector_type, selector_value in price_selectors:
        try:
            price_element = WebDriverWait(driver, CONFIG["WAIT_TIMEOUT"]).until(
                EC.presence_of_element_located((selector_type, selector_value))
            )
            found_price = price_element.text
            time.sleep(random.uniform(CONFIG["MIN_DELAY"], CONFIG["MIN_DELAY"]+2))
            return found_price
        except TimeoutException:
            continue
    
    return "정보 없음"

def get_rating(driver):
    """평점 정보 수집"""
    rating_selectors = [
        (By.CSS_SELECTOR, ".rating"),
        (By.CSS_SELECTOR, "[class*='rating']"),
        (By.XPATH, "//span[contains(@class, 'rating')]"),
        (By.XPATH, "/html/body/div[1]/main/div[1]/section/div[1]/span/span[2]")
    ]

    for selector_type, selector_value in rating_selectors:
        try:
            rating_element = WebDriverWait(driver, CONFIG["WAIT_TIMEOUT"]).until(
                EC.presence_of_element_located((selector_type, selector_value))
            )
            found_rating = rating_element.text
            time.sleep(random.uniform(2, 4))
            return found_rating
        except TimeoutException:
            continue
    
    return "정보 없음"

def get_review_count(driver):
    """리뷰 수 정보 수집"""
    print(f"  📝 리뷰 수 정보 찾는 중...")
    review_count_selectors = [
        (By.XPATH, "//span[contains(text(), '리뷰')]"),
        (By.XPATH, "//span[contains(text(), 'review')]"),
        (By.XPATH, "//span[contains(text(), '후기')]"),
        (By.XPATH, "//span[contains(text(), '개')]"),
        (By.XPATH, "//span[contains(text(), '건')]"),
    ]

    for selector_type, selector_value in review_count_selectors:
        try:
            review_element = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((selector_type, selector_value))
            )
            review_text = review_element.text.strip()
            
            review_keywords = ['리뷰', '후기', 'review', '개', '건']
            has_number = any(char.isdigit() for char in review_text)
            has_keyword = any(keyword in review_text.lower() for keyword in review_keywords)
            
            if has_number and has_keyword and len(review_text) < 50:
                print(f"  ✅ 리뷰 수 정보 발견: {review_text}")
                return review_text
                
        except TimeoutException:
            continue

    print(f"  ℹ️ 리뷰 수 정보를 찾을 수 없습니다.")
    return ""

def get_language(driver):
    """언어 정보 수집"""
    print(f"  🌐 언어 정보 찾는 중...")
    language_selectors = [
        (By.XPATH, "//span[contains(text(), '언어')]"),
        (By.XPATH, "//span[contains(text(), '한국어')]"),
        (By.XPATH, "//span[contains(text(), '영어')]"),
        (By.XPATH, "//span[contains(text(), 'Korean')]"),
        (By.XPATH, "//span[contains(text(), 'English')]"),
    ]

    for selector_type, selector_value in language_selectors:
        try:
            language_element = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((selector_type, selector_value))
            )
            language_text = language_element.text.strip()
            
            language_keywords = ['언어', '한국어', '영어', '중국어', '일본어', 'Korean', 'English']
            
            if any(keyword in language_text for keyword in language_keywords):
                print(f"  ✅ 언어 정보 발견: {language_text}")
                return language_text
                
        except TimeoutException:
            continue

    print(f"  ℹ️ 언어 정보를 찾을 수 없습니다.")
    return ""

def download_image(driver, product_name, city_name):
    """이미지 다운로드"""
    if not CONFIG["SAVE_IMAGES"]:
        return "이미지 저장 비활성화"
        
    print(f"  🖼️ 상품 이미지 다운로드 중...")
    image_selectors = [
        (By.CSS_SELECTOR, ".product-image img"),
        (By.CSS_SELECTOR, ".gallery img:first-child"),
        (By.XPATH, "//img[contains(@alt, '상품')]"),
    ]

    img_url = None
    for selector_type, selector_value in image_selectors:
        try:
            img_element = WebDriverWait(driver, CONFIG["WAIT_TIMEOUT"]).until(
                EC.presence_of_element_located((selector_type, selector_value))
            )
            img_url = img_element.get_attribute('src')
            break
        except TimeoutException:
            continue

    if img_url:
        try:
            safe_product_name_raw = str(product_name) if product_name else "상품명없음"
            safe_product_name = make_safe_filename(f"{city_name}_{safe_product_name_raw[:CONFIG['MAX_PRODUCT_NAME_LENGTH']]}")
            img_download_path = os.path.abspath("") + "/myrealtripthumb_img/" + safe_product_name + ".png"
            urllib.request.urlretrieve(img_url, img_download_path)
            print(f"  ✅ 이미지 다운로드 완료!")
            return img_download_path
        except Exception as e:
            print(f"  ⚠️ 이미지 다운로드 실패: {type(e).__name__}: {e}")
            return "다운로드 실패"
    else:
        return "이미지 없음"

def crawl_all_products_in_page(driver, city_name, continent, country):
    """1페이지에서 설정된 개수만큼 상품을 순차적으로 크롤링하는 함수"""
    print(f"\n🎯 {city_name} - 1페이지에서 {CONFIG['MAX_PRODUCTS_PER_CITY']}개 상품 크롤링 시작!")
    
    # URL 수집
    try:
        product_urls = retry_operation(
            lambda: collect_page_urls(driver), 
            "상품 URL 수집"
        )
        
        if not product_urls:
            print(f"  ❌ {city_name}: 상품 URL을 찾을 수 없습니다.")
            return []
            
    except Exception as e:
        print(f"  ❌ {city_name}: URL 수집 실패 - {type(e).__name__}")
        return []
    
    # 설정된 개수만큼만 선택
    max_products = CONFIG["MAX_PRODUCTS_PER_CITY"]
    selected_urls = product_urls[:max_products]
    
    print(f"  📊 총 {len(product_urls)}개 상품 중 {len(selected_urls)}개를 크롤링합니다!")
    
    page_results = []
    total_products = len(selected_urls)
    
    print("  " + "="*60)
    
    for product_index, product_url in enumerate(selected_urls, 1):
        print_product_progress(product_index, total_products, f"상품 {product_index}")
        
        # 기본값 설정
        product_name = "정보 없음"
        price = "정보 없음"
        grade_review = "정보 없음"
        review_count = ""
        language = ""
        img_path = "처리 안됨"
        current_url = product_url
        
        try:
            # 상품 페이지로 이동
            print(f"    🔗 상품 {product_index} URL로 이동 중...")
            driver.get(product_url)
            time.sleep(random.uniform(CONFIG["MIN_DELAY"], CONFIG["MAX_DELAY"]))
            
            # 정보 수집
            try:
                product_name = retry_operation(
                    lambda: get_product_name(driver), 
                    f"상품 {product_index} 이름 수집"
                )
            except Exception as e:
                print(f"    ⚠️ 상품명 수집 실패: {type(e).__name__}")
                product_name = f"수집실패_{product_index}"

            try:
                price = get_price(driver)
            except Exception as e:
                print(f"    ⚠️ 가격 정보 수집 실패: {type(e).__name__}")
                price = "정보 없음"

            try:
                grade_review = get_rating(driver)
            except Exception as e:
                print(f"    ⚠️ 평점 정보 수집 실패: {type(e).__name__}")
                grade_review = "정보 없음"

            try:
                review_count = get_review_count(driver)
            except Exception as e:
                print(f"    ⚠️ 리뷰 수 정보 수집 실패: {type(e).__name__}")
                review_count = ""

            try:
                language = get_language(driver)
            except Exception as e:
                print(f"    ⚠️ 언어 정보 수집 실패: {type(e).__name__}")
                language = ""

            try:
                img_path = download_image(driver, product_name, city_name)
            except Exception as e:
                print(f"    ⚠️ 이미지 처리 실패: {type(e).__name__}")
                img_path = "처리 실패"

            # 결과 저장
            result = {
                '번호': len(page_results) + 1,
                '대륙': continent,
                '국가': country,
                '도시': city_name,
                '상품번호': product_index,
                '상품명': product_name,
                '가격': price,
                '평점': grade_review,
                '리뷰수': review_count,
                '언어': language,
                '이미지_경로': img_path,
                'URL': current_url,
                '수집_시간': time.strftime('%Y-%m-%d %H:%M:%S'),
                '상태': '완전수집'
            }

            page_results.append(result)

            # 상품 정보 출력
            safe_name = str(product_name)[:40] + "..." if len(str(product_name)) > 40 else str(product_name)
            print(f"    ✅ 상품 {product_index} 크롤링 완료!")
            print(f"       상품명: {safe_name}")
            print(f"       가격: {price}")
            print(f"       평점: {grade_review}")
            print(f"       리뷰수: {review_count if review_count else '정보 없음'}")
            print(f"       언어: {language if language else '정보 없음'}")
            
            # 다음 상품을 위한 휴식
            if product_index < total_products:
                wait_time = random.uniform(2, 5)
                print(f"    ⏰ 다음 상품까지 {wait_time:.1f}초 대기...")
                time.sleep(wait_time)

        except Exception as e:
            print(f"    ❌ 상품 {product_index}: 예상치 못한 오류 - {type(e).__name__}")
            continue
    
    print(f"\n  🎉 {city_name} - {len(selected_urls)}개 상품 크롤링 완료!")
    print(f"  📊 총 {len(page_results)}개 상품 정보를 수집했습니다.")
    print("  " + "="*60)
    
    return page_results

print("✅ 그룹 1 완료: 모든 함수가 정의되었습니다!")
print(f"🔢 현재 설정: {CONFIG['MAX_PRODUCTS_PER_CITY']}개 상품 크롤링")
print(f"🏙️ 검색 도시: {CITIES_TO_SEARCH}")
print("🎯 다음: 그룹 2를 실행하세요!")




🔧 Selenium 버전: 4.25.0
✅ CITY_CODES 추가 완료! 30개 도시 지원
🔧 Phase 2: 확장성 개선 시스템 초기화...
⚙️ CONFIG 확장성 설정 업데이트 완료!
📝 city_codes.json 파일이 없어서 새로 생성합니다...
✅ city_codes.json 파일 생성 완료! (30개 도시)
🔄 CITY_CODES 업데이트: 30개 도시
✅ Phase 2 시스템 초기화 완료!
🚀 자주 사용하는 도시들 추가 중...
✅ 메모리에 추가: 교토 → KIX
✅ 메모리에 추가: 인천 → ICN
✅ 메모리에 추가: 김포 → GMP
✅ 메모리에 추가: 하와이 → HNL
✅ 메모리에 추가: 괌 → GUM
✅ 메모리에 추가: 사이판 → SPN
✅ 메모리에 추가: 푸꾸옥 → PQC
✅ 메모리에 추가: 나트랑 → CXR
✅ 메모리에 추가: 보홀 → TAG
✅ 메모리에 추가: 랑카위 → LGK
✅ city_codes.json 파일 생성 완료! (40개 도시)
✅ 10개 도시 일괄 추가 완료!

🌍 지원하는 도시 목록:

📍 동남아시아:
   방콕 → BKK
   치앙마이 → CNX
   푸켓 → HKT
   싱가포르 → SIN
   홍콩 → HKG
   쿠알라룸푸르 → KUL
   세부 → CEB
   다낭 → DAD
   호치민 → SGN

📍 일본:
   도쿄 → NRT
   오사카 → KIX
   나고야 → NGO
   후쿠오카 → FUK
   오키나와 → OKA
   삿포로 → CTS

📍 한국:
   서울 → ICN
   부산 → PUS
   제주 → CJU
   대구 → TAE
   광주 → KWJ
   여수 → RSU

📍 유럽:
   파리 → CDG
   런던 → LHR
   로마 → FCO
   바르셀로나 → BCN

📍 북미:
   뉴욕 → JFK
   로스앤젤레스 → LAX
   시카고 → ORD

📍 오세아니아:
   시드니 → SYD
   멜버른 → MEL

📊 총 40개 도시 지원

🎉 Phase 2: 확장성 개선 완료!
💡

In [2]:
# 🚀 그룹 2: 도시 하드코딩 해결 - 드라이버 실행 + 도시 검색
# (이 셀을 실행하면 브라우저가 열리고 그룹 1에서 설정한 도시 검색까지 완료됩니다)

print("🔄 마이리얼트립 크롤링 시작!")

# 결과 저장소 초기화
all_results = []
print("🔄 결과 저장소 초기화 완료")

# 크롬 드라이버 실행
try:
    driver = setup_driver()
except Exception as e:
    print(f"❌ 드라이버 설정 실패: {e}")
    print("⚠️ 그룹 1을 먼저 실행했는지 확인하세요!")

# 썸네일 폴더 생성
if CONFIG["SAVE_IMAGES"]:
    img_folder_path = os.path.abspath("") + "/myrealtripthumb_img"
    try:
        shutil.rmtree(img_folder_path)
    except:
        pass
    os.makedirs(img_folder_path, exist_ok=True)
    print("📁 이미지 폴더 생성 완료")

# 🆕 그룹 1 설정에서 도시 가져오기
if not CITIES_TO_SEARCH:
    print("❌ CITIES_TO_SEARCH가 비어있습니다!")
    print("💡 그룹 1에서 CITIES_TO_SEARCH = ['도시명']을 설정하세요!")
    raise ValueError("검색할 도시가 설정되지 않았습니다")

city_name = CITIES_TO_SEARCH[0]  # 🆕 첫 번째 도시 사용
continent, country = get_city_info(city_name)

print("="*60)
print(f"🌏 그룹 1 설정 도시: {city_name} 검색 시작!")
print(f"⚙️  설정: 재시도 {CONFIG['RETRY_COUNT']}회, 타임아웃 {CONFIG['WAIT_TIMEOUT']}초")
print("="*60)

print(f"🔍 진행률: [░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░] 0.0% (0/1)")
print(f"📍 현재 작업: {city_name} - 진행중")
print("-" * 50)
print(f"  🌍 대륙: {continent} | 국가: {country}")

try:
    # 1. 메인 페이지로 이동
    print("  📱 마이리얼트립 메인 페이지 이동 중...")
    retry_operation(
        lambda: go_to_main_page(driver), 
        "메인 페이지 이동"
    )
    print(f"  ✅ 마이리얼트립 페이지 열기 완료")
    
    # 2. 검색창 찾기 및 입력 (🆕 동적 도시명 사용)
    retry_operation(
        lambda: find_and_fill_search(driver, city_name), 
        f"{city_name} 검색창 입력"
    )

    # 3. 검색 버튼 클릭
    retry_operation(
        lambda: click_search_button(driver), 
        "검색 버튼 클릭"
    )

    # 4. 팝업 처리 (선택사항)
    try:
        handle_popup(driver)
    except Exception as e:
        print(f"  ℹ️ 팝업 처리 중 오류 (무시됨): {type(e).__name__}")

    # 5. 전체 상품 보기 클릭 (선택사항)
    try:
        click_view_all(driver)
    except Exception as e:
        print(f"  ℹ️ 전체 상품 보기 처리 중 오류 (무시됨): {type(e).__name__}")

    print(f"\n✅ 그룹 2 완료: {city_name} 검색 성공!")
    print(f"🎯 현재 상태: {city_name} 상품 목록 페이지에 있습니다")
    print("🚀 다음: 그룹 3을 실행하여 URL 수집 테스트하세요!")
    
except TimeoutException as e:
    print(f"  ⏰ {city_name}: 페이지 로딩 시간 초과")
    print(f"  📍 위치: {continent} > {country} > {city_name}")
    print("❌ 그룹 2 실패: 시간 초과")
    
except NoSuchElementException as e:
    print(f"  🔍 {city_name}: 필요한 웹 요소를 찾을 수 없음")
    print(f"  📍 위치: {continent} > {country} > {city_name}")
    print("❌ 그룹 2 실패: 요소 없음")
    
except Exception as e:
    print(f"  ❌ {city_name}: 예상치 못한 오류 - {type(e).__name__}: {e}")
    print("❌ 그룹 2 실패: 예상치 못한 오류")
    print("💡 그룹 1을 먼저 실행했는지 확인하세요!")
    print(f"💡 CITIES_TO_SEARCH에 '{city_name}'이 올바르게 설정되었는지 확인하세요!")
    

🔄 마이리얼트립 크롤링 시작!
🔄 결과 저장소 초기화 완료
❌ 드라이버 설정 실패: 'USER_AGENT'
⚠️ 그룹 1을 먼저 실행했는지 확인하세요!
📁 이미지 폴더 생성 완료
🌏 그룹 1 설정 도시: 오사카 검색 시작!
⚙️  설정: 재시도 3회, 타임아웃 10초
🔍 진행률: [░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░] 0.0% (0/1)
📍 현재 작업: 오사카 - 진행중
--------------------------------------------------
  🌍 대륙: 아시아 | 국가: 일본
  📱 마이리얼트립 메인 페이지 이동 중...
  ❌ 메인 페이지 이동 예상치 못한 오류: NameError: name 'driver' is not defined
  ❌ 오사카: 예상치 못한 오류 - NameError: name 'driver' is not defined
❌ 그룹 2 실패: 예상치 못한 오류
💡 그룹 1을 먼저 실행했는지 확인하세요!
💡 CITIES_TO_SEARCH에 '오사카'이 올바르게 설정되었는지 확인하세요!


In [ ]:
# 🔍 그룹 3: URL 수집 테스트 (24개 완전 수집 버전)
# (이 셀을 실행하면 현재 페이지에서 Products + Offers 24개 URL을 모두 수집합니다)

print("🔍 그룹 3: URL 수집 테스트 시작! (24개 완전 수집 버전)")
print("="*50)

def collect_all_24_urls(driver):
    print(f"  🔍 Products와 Offers URL 모두 수집 중...")
    
    # 페이지 로딩 완료 대기
    time.sleep(5)
    
    all_links = driver.find_elements(By.TAG_NAME, "a")
    print(f"  📊 전체 링크 수: {len(all_links)}개")
    
    products_urls = []
    offers_urls = []
    
    for link in all_links:
        try:
            href = link.get_attribute('href')
            if href:
                if 'experiences.myrealtrip.com/products/' in href:
                    if href not in products_urls:
                        products_urls.append(href)
                elif 'myrealtrip.com/offers/' in href:
                    if href not in offers_urls:
                        offers_urls.append(href)
        except:
            continue
    
    # 두 리스트 합치기 (products 먼저, offers 나중에)
    all_urls = products_urls + offers_urls
    
    print(f"  ✅ Products URL: {len(products_urls)}개")
    print(f"  ✅ Offers URL: {len(offers_urls)}개")
    print(f"  🎉 총 수집: {len(all_urls)}개")
    
    return all_urls

try:
    # 24개 URL 수집 실행
    print("📊 현재 페이지에서 24개 상품/오퍼 URL 수집을 시작합니다...")
    
    collected_urls = collect_all_24_urls(driver)
    
    if collected_urls:
        print(f"\n🎉 URL 수집 성공!")
        print(f"📈 총 {len(collected_urls)}개의 상품/오퍼 URL을 찾았습니다!")
        
        # 설정된 개수와 비교
        max_products = CONFIG['MAX_PRODUCTS_PER_CITY']
        will_crawl = min(len(collected_urls), max_products)
        
        print(f"🔢 설정된 크롤링 개수: {max_products}개")
        print(f"🎯 실제 크롤링할 개수: {will_crawl}개")
        
        print(f"\n📋 수집된 URL 목록:")
        print("-" * 80)
        for i, url in enumerate(collected_urls, 1):
            url_type = "🛍️ Product" if "/products/" in url else "🏷️ Offer"
            print(f"  {i:2d}. {url_type}: {url}")
        
        # URL 유효성 체크
        print(f"\n🔍 URL 유효성 체크:")
        products_count = sum(1 for url in collected_urls if '/products/' in url)
        offers_count = sum(1 for url in collected_urls if '/offers/' in url)
        
        print(f"  ✅ Products: {products_count}개")
        print(f"  ✅ Offers: {offers_count}개")
        print(f"  🎯 총 유효 URL: {products_count + offers_count}개")
        
        if len(collected_urls) >= 20:
            print("\n🚀 URL 수집 대성공! 다음 단계로 진행 가능합니다!")
            print("✅ 그룹 3 완료: 24개 URL 수집 테스트 성공")
            print("🎯 다음: 그룹 4를 실행하여 24개 상품/오퍼 크롤링을 시작하세요!")
        else:
            print("\n⚠️ 수집된 URL이 예상보다 적습니다.")
            
    else:
        print("❌ URL 수집 실패: 상품/오퍼 URL을 찾을 수 없습니다")
        print("💡 추가 디버깅:")
        
        # 디버깅 정보
        try:
            page_source = driver.page_source
            products_count = page_source.count('experiences.myrealtrip.com/products/')
            offers_count = page_source.count('myrealtrip.com/offers/')
            print(f"   📄 페이지 소스에서 products 패턴: {products_count}회")
            print(f"   📄 페이지 소스에서 offers 패턴: {offers_count}회")
            
        except Exception as debug_error:
            print(f"   ❌ 디버깅 실패: {debug_error}")
        
        print("🔄 그룹 2를 다시 실행해보세요!")
        
except NameError as e:
    print(f"❌ 변수 오류: {e}")
    print("💡 그룹 1과 그룹 2를 먼저 실행했는지 확인하세요!")
    
except Exception as e:
    print(f"❌ URL 수집 중 오류: {type(e).__name__}: {e}")
    print("💡 가능한 해결책:")
    print("   1. 그룹 2가 성공적으로 완료되었는지 확인")
    print("   2. 브라우저가 방콕 상품 목록 페이지에 있는지 확인")
    print("   3. 인터넷 연결 상태 확인")

print("\n" + "="*50)

In [ ]:
# 🛡️ 10개 크롤링용 안전장치 - 그룹 4 수정
# 기존 그룹 4의 크롤링 루프 부분을 이것으로 교체하세요!

print("🚀 안전장치 적용된 10개 상품 크롤링!")
print("="*80)

# 그룹 1 설정에서 도시 가져오기  
if not CITIES_TO_SEARCH:
    print("❌ CITIES_TO_SEARCH가 비어있습니다!")
    raise ValueError("검색할 도시가 설정되지 않았습니다")

city_name = CITIES_TO_SEARCH[0]
total_products = min(CONFIG['MAX_PRODUCTS_PER_CITY'], len(collected_urls))

print(f"📊 그룹 1 설정값: {CONFIG['MAX_PRODUCTS_PER_CITY']}개 상품")
print(f"🎯 실제 크롤링 대상: {total_products}개 상품")
print(f"🏙️ 도시: {city_name}")
print(f"⏱️ 안전 지연시간: {CONFIG['MIN_DELAY']}-{CONFIG['MAX_DELAY']}초")
print(f"🛡️ 중간 저장: {CONFIG['MEMORY_CLEANUP_INTERVAL']}개마다")

# 도시 정보 가져오기
continent, country = get_city_info(city_name)
city_code = get_city_code(city_name)

print(f"🌍 위치: {continent} > {country} > {city_name}")
print(f"✈️ 공항 코드: {city_code}")

# 크롤링할 URL 목록 출력
print(f"\n📋 크롤링할 상품 목록:")
urls_to_crawl = collected_urls[:total_products]

for i, url in enumerate(urls_to_crawl, 1):
    url_type = "Product" if "/products/" in url else "Offer"
    img_filename = f"{city_code}_{i:03d}.jpg"
    print(f"  {i}. 🛍️ {url_type}: {url}")
    print(f"     📷 이미지: {img_filename}")

print("="*80)

# 🛡️ 안전장치 변수들 초기화
all_results = []
intermediate_results = []  # 중간 저장용
failed_products = []       # 실패한 상품들
memory_cleanup_count = 0

# 시작 시간 기록
import time
start_time = time.time()

# 🔄 안전한 크롤링 루프
for product_index, product_url in enumerate(urls_to_crawl, 1):
    try:
        print_product_progress(product_index, total_products, f"상품 {product_index}")
        
        print(f"🔗 상품 {product_index} URL로 이동 중...")
        print(f"📍 {product_url}")
        
        # 🛡️ 안전장치: 더 긴 대기시간
        if CONFIG.get("LONGER_DELAYS", False):
            delay = random.uniform(CONFIG['MIN_DELAY'], CONFIG['MAX_DELAY'])
            print(f"⏱️ 안전 대기: {delay:.1f}초")
            time.sleep(delay)
        
        # URL로 이동
        driver.get(product_url)
        
        # 페이지 로딩 확인
        if driver.current_url != product_url:
            print(f"⚠️ URL 변경됨: {driver.current_url}")
        else:
            print(f"✅ 정상 이동 완료")
        
        # 🛡️ 안전장치: 페이지 안정화 대기
        time.sleep(3)
        
        print(f"📊 상품 정보 수집 중...")
        
        # 상품 정보 수집
        try:
            product_name = get_product_name_by_type(driver, "Product" if "/products/" in product_url else "Offer")
            print(f"✅ 상품명 수집 완료")
        except Exception as e:
            print(f"⚠️ 상품명 수집 실패: {type(e).__name__}")
            product_name = f"수집실패_{product_index}"
        
        try:
            price_raw = get_price_fixed(driver)
            price_clean = extract_clean_price(price_raw)
            print(f"✅ 가격 수집 완료")
        except Exception as e:
            print(f"⚠️ 가격 수집 실패: {type(e).__name__}")
            price_raw, price_clean = "정보 없음", "정보 없음"
        
        try:
            rating_raw = get_rating(driver)
            rating_clean = extract_clean_rating(rating_raw)
            print(f"✅ 평점 수집 완료")
        except Exception as e:
            print(f"⚠️ 평점 수집 실패: {type(e).__name__}")
            rating_raw, rating_clean = "정보 없음", "정보 없음"
        
        try:
            review_count = get_review_count(driver)
            print(f"✅ 리뷰수 수집 완료")
        except Exception as e:
            print(f"⚠️ 리뷰수 수집 실패: {type(e).__name__}")
            review_count = "정보 없음"
        
        try:
            language_info = get_language(driver)
            print(f"✅ 언어 수집 완료")
        except Exception as e:
            print(f"⚠️ 언어 수집 실패: {type(e).__name__}")
            language_info = "정보 없음"
        
        # 이미지 다운로드
        try:
            img_info = download_image_improved_fixed(driver, product_name, city_name, product_index)
            print(f"✅ 이미지 다운로드 완료")
        except Exception as e:
            print(f"⚠️ 이미지 다운로드 실패: {type(e).__name__}")
            img_info = {
                'status': '다운로드 실패',
                'filename': f'{city_code}_{product_index:03d}.jpg',
                'path': '',
                'size': 0
            }
        
        # 결과 저장
        result = {
            '상품번호': product_index,
            '상품명': product_name,
            '가격_원본': price_raw,
            '가격_정제': price_clean,
            '평점_원본': rating_raw,
            '평점_정제': rating_clean,
            '리뷰수': review_count,
            '언어': language_info,
            '대륙': continent,
            '국가': country,
            '도시': city_name,
            '공항코드': city_code,
            '사이트': '마이리얼트립',
            '사이트_코드': 'MRT',
            '상품타입': 'Product' if '/products/' in product_url else 'Offer',
            'URL': product_url,
            '이미지_파일명': img_info['filename'],
            '이미지_경로': img_info['path'],
            '이미지_상태': img_info['status'],
            '이미지_크기': img_info['size'],
            '수집_시간': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            '수집_상태': '완전수집'
        }
        
        all_results.append(result)
        intermediate_results.append(result)
        
        print(f"💾 상품 {product_index} 결과 저장 완료\n")
        
        # 🛡️ 중간 저장 체크포인트
        if product_index % CONFIG.get('MEMORY_CLEANUP_INTERVAL', 5) == 0:
            print("🔄 중간 저장 및 메모리 정리 중...")
            
            # 중간 결과 저장
            if intermediate_results:
                intermediate_df = pd.DataFrame(intermediate_results)
                intermediate_filename = f"temp_중간저장_{city_name}_{product_index}개_{datetime.now().strftime('%H%M%S')}.csv"
                intermediate_df.to_csv(intermediate_filename, index=False, encoding='utf-8-sig')
                print(f"💾 중간 저장 완료: {intermediate_filename}")
                
                # 중간 결과 리스트 초기화
                intermediate_results = []
            
            # 🛡️ 브라우저 메모리 정리
            try:
                driver.execute_script("window.sessionStorage.clear();")
                driver.execute_script("window.localStorage.clear();")
                print("🧹 브라우저 메모리 정리 완료")
                memory_cleanup_count += 1
            except Exception as e:
                print(f"⚠️ 메모리 정리 실패: {e}")
            
            # 진행 상황 출력
            elapsed_time = time.time() - start_time
            avg_time = elapsed_time / product_index
            remaining_products = total_products - product_index
            estimated_remaining = avg_time * remaining_products
            
            print(f"📊 진행 상황:")
            print(f"   ✅ 완료: {product_index}/{total_products}개")
            print(f"   ⏱️ 소요시간: {elapsed_time/60:.1f}분")
            print(f"   🕐 예상 남은시간: {estimated_remaining/60:.1f}분")
            print(f"   🧹 메모리 정리: {memory_cleanup_count}회")
            print()
        
    except Exception as e:
        print(f"❌ 상품 {product_index} 오류: {type(e).__name__}: {str(e)}")
        
        # 실패한 상품 기록
        failed_product = {
            '상품번호': product_index,
            'URL': product_url,
            '오류': str(e),
            '시간': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }
        failed_products.append(failed_product)
        
        # 부분 결과라도 저장
        partial_result = {
            '상품번호': product_index,
            '상품명': f'오류발생_{product_index}',
            '가격_원본': '수집실패',
            '가격_정제': '수집실패',
            '평점_원본': '수집실패',
            '평점_정제': '수집실패',
            '리뷰수': '수집실패',
            '언어': '수집실패',
            '대륙': continent,
            '국가': country,
            '도시': city_name,
            '공항코드': city_code,
            '사이트': '마이리얼트립',
            '사이트_코드': 'MRT',
            '상품타입': 'Product' if '/products/' in product_url else 'Offer',
            'URL': product_url,
            '이미지_파일명': f'{city_code}_{product_index:03d}.jpg',
            '이미지_경로': '다운로드실패',
            '이미지_상태': '다운로드실패',
            '이미지_크기': 0,
            '수집_시간': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            '수집_상태': '부분수집'
        }
        
        all_results.append(partial_result)
        
        print(f"⚠️ 상품 {product_index} 부분 결과 저장\n")
        
        # 오류 후 더 긴 대기
        print("⏱️ 오류 복구 대기 중...")
        time.sleep(10)

# 🎯 최종 결과 저장 및 통계
total_time = time.time() - start_time
successful_products = len([r for r in all_results if r['수집_상태'] == '완전수집'])
failed_count = len(failed_products)

print("🏁 크롤링 완료!")
print("="*80)
print(f"📊 최종 통계:")
print(f"   ✅ 성공: {successful_products}개")
print(f"   ❌ 실패: {failed_count}개")
print(f"   📈 성공률: {(successful_products/total_products)*100:.1f}%")
print(f"   ⏱️ 총 소요시간: {total_time/60:.1f}분")
print(f"   📊 평균 처리시간: {total_time/total_products:.1f}초/상품")
print(f"   🧹 메모리 정리: {memory_cleanup_count}회")

# 최종 CSV 저장
if all_results:
    save_myrealtrip_data(all_results)
    print(f"📁 최종 CSV 저장 완료!")

# 실패한 상품들 정보 저장
if failed_products:
    failed_df = pd.DataFrame(failed_products)
    failed_filename = f"failed_products_{city_name}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    failed_df.to_csv(failed_filename, index=False, encoding='utf-8-sig')
    print(f"📁 실패 목록 저장: {failed_filename}")

print("🎉 안전장치 적용된 10개 크롤링 완료!")